In [ ]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Saving the Infidels

In this notebook we want so solve a famous search problem, which is usually known as the
[missionaries and cannibals problem](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem):
Three missinaries and three infidels have to cross a river in order to get to a church where the infidels can be baptized.  In order to cross the river, they have to take a small boat that can take at most two passengers.  If at any moments at any shore there are more infidels than missionaries, then the missionaries have a problem, since the infidels have a diet that includes human flesh.

We will encode this problem as a *constraint satisfaction problem*.  In order to do so, we assume that the
problem can be solved with $n\in\mathbb{N}$ crossing of the river.  We use the following variables:
* $\texttt{M}i$ for $i\in\{0,\cdots,n\}$ is the number of missionaries on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{C}i$ for $i\in\{0,\cdots,n\}$ is the number of infidels on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{MB}i$ for $i\in\{1,\cdots,n\}$ is the number of missionaries that cross the river
  during the $i^{\textrm{th}}$ crossing.
* $\texttt{CB}i$ for $i\in\{1,\cdots,n\}$ is the number of infidels that cross the river
  during the $i^{\textrm{th}}$ crossing.

## Auxiliary Functions

The function `flatten` takes a list of lists `LoL` and returns a list containing all the elements contained in any of the lists in `LoL`.

In [ ]:
def flatten(LoL):
    return [x for L in LoL for x in L]

In [ ]:
flatten([[1,2], [3,4]])

The function `no_problem(i)` takes an integer `i` of as its input.
It returns a formula that is true if there is no problem on either shore of the river after the $i^{\textrm{th}}$ crossing.  There is no problem if any of the following conditions is true:
* There are no missionaries on the western side of the shore, i.e. $\texttt{M}i = 0$.  
  Then all missionaries are on the eastern side of the shore.
* All missionaries are on the western side of the shore, i.e. $\texttt{M}i = 3$.
  Then there are no missionaries are on the eastern side of the shore.
* The number of missionaries on the western side is the same as the number of infidels on that side,
  i.e. $\texttt{M}i = \texttt{C}i$.  Then these numbers have to match on the eastern shore as well.

In [ ]:
def no_problem(i):
    return f'M{i} == 0 or M{i} == 3 or M{i} == C{i}'

In [ ]:
no_problem(7)

The function `boat_ok` is true if the boat is neither empty nor overloaded during the $i^{\textrm{th}}$ crossing, i.e. the number of passengers is either 1 or 2.

In [ ]:
def boat_ok(i):
    return f'1 <= MB{i} + CB{i} <= 2'

In [ ]:
boat_ok(3)

The function `change_ok(i)` returns a set of formulas that is true if the missionaries starting on one shore arrive at the opposite  shore after the $i^{\textrm{th}}$ crossing.  Note that if $i$ is odd, then during the $i^{\textrm{th}}$ crossing the boat travels from the western shore to the eastern shore.  If $i$ is even, the boat travels from the eastern shore to the western shore.

In [ ]:
def change_ok(i):
    if i % 2 == 0:
        return { f'M{i+1} + MB{i+1} == M{i}', f'C{i+1} + CB{i+1} == C{i}'}
    else:
        return { f'M{i} + MB{i+1} == M{i+1}', f'C{i} + CB{i+1} == C{i+1}'}

In [ ]:
change_ok(0)

In [ ]:
change_ok(1)

The function `missionaries` creates a CSP that tries to solve the problem with `n` crossings.

In [ ]:
def missionaries_CSP(n):
        "Returns a CSP encoding the problem."
        Lists        = [[f'MB{i}', f'CB{i}', f'M{i}', f'C{i}'] for i in range(n+1)]
        Variables    = flatten(Lists)
        Variables    = Variables[2:] # remove MB0 and CB0
        Values       = { 0, 1, 2, 3 }
        Constraints  = { 'M0 == 3', 'C0 == 3', f'M{n} == 0', f'C{n} == 0' }
        Constraints |= { no_problem(i) for i in range(n) }
        Constraints |= { boat_ok(i)    for i in range(1, n+1) }
        for i in range(n):
            Constraints |= change_ok(i)
        return Variables, Values, Constraints

In [ ]:
missionaries_CSP(1)

In [ ]:
%run Backtracking-Constraint-Solver.ipynb

The function `find_solution` computes a solution to the problem of saving the infidels.

In [ ]:
def find_solution():
    n = 1
    while True:
        print(n)
        CSP = missionaries_CSP(n)
        Solution = solve(CSP)
        if Solution != None:
            return n, Solution
        n += 2

On my desktop computer (2017 iMac with 3.4 GHz Quad-Core Intel i5) it takes about 2 seconds to solve the problem. 

In [ ]:
%%time
n, Solution = find_solution()
n, Solution

In [ ]:
def show_solution(Solution, n):
    for i in range(n+1):
        M  = Solution[f'M{i}']
        C  = Solution[f'C{i}']
        if i + 1 < n:
            MB = Solution[f'MB{i+1}']
            CB = Solution[f'CB{i+1}']
        print('😇' * M + '🥷' * C + ' ' * 28 + '😇' * (3 - M) + '🥷' * (3 - C))
        if i % 2 == 0:
            print(' ' * 12 + '>>> ' + '😇'*MB + '🥷'*CB + ' >>>')
        elif i % 2 == 1 and i + 1 < n:
            print(' ' * 12 + '<<< ' + '😇'*MB + '🥷'*CB + ' <<<')

In [ ]:
show_solution(Solution, n)